In [2]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

: 

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.util.Enum
import chisel3.tester.RawTester.test

cmd2.sc:1: not found: value chisel3
import chisel3._
       ^cmd2.sc:2: not found: value chisel3
import chisel3.util._
       ^cmd2.sc:3: not found: value chisel3
import chisel3.tester._
       ^cmd2.sc:4: not found: value chisel3
import chisel3.util.Enum
       ^cmd2.sc:5: not found: value chisel3
import chisel3.tester.RawTester.test
       ^Compilation Failed

: 

In [11]:
class Memory extends Module {
    val io = IO(new Bundle {
        val in = Flipped(Decoupled())
        val data_in = Input(SInt(32.W))
        val addr_in = Input(UInt(5.W))
        val write_en = Input(Bool())
        val out = Decoupled()
        val data_out = Output(SInt(32.W))
    })
    
    // Memory Array of 32, 32-bit elements
    val mem = Mem(32, SInt())
    
    object State extends Enum {
        val IDLE, READ, WRITE  = UInt(2.W) // i "load"  -> 000_0011
    }
    
    // Define default outputs
    io.data_out := WireDefault(io.data_in)
    io.out.ready := WireDefault(false.B)
    val state = RegInit(State.IDLE)
    
    //
    switch(state) {
        is (State.IDLE) {
            when (io.in.valid) {
                when (io.write_en) {
                    state := State.WRITE
                } .otherwise {
                    state := State.READ
                }
            } .otherwise {
                state := State.IDLE
            }
        }
        is (State.READ) {
            when (io.in.valid) {
                io.data_out := mem(io.addr_in)
                io.out.ready := true.B
                state := State.IDLE
            } .otherwise {
                state := State.IDLE
            }
            
        }
        is (State.WRITE) {
            when (io.in.valid) {
                mem(io.addr_in) := io.data_in
                io.out.ready := true.B
                state := State.IDLE
            } .otherwise {
                state := State.IDLE
            }
        }
    }
}


println(getVerilog(new Memory))

Elaborating design...
[error] 	at ... ()
[error] 	at ammonite.$sess.cmd10$Helper$Memory.<init>(cmd10.sc:20)
[error] 	at ammonite.$sess.cmd10$Helper.$anonfun$res10_1$1(cmd10.sc:59)
[error] 	at ... ()
[error] 	at ... (Stack trace trimmed to user code only. Rerun with --full-stacktrace to see the full stack trace)


: 

In [ ]:
test(new Memory) { c =>
    
    c.io.addr_in.poke(0.U)
    c.io.data_in.poke(1.S)
    c.io.write_en.poke(true.B)
    c.clock.step(1)
    c.io.in.valid.poke(true.B)
    c.clock.step(1)
    
    
//     c.io.in.poke(0.U)     // Set our input to value 0
//     c.clock.step(1)    // advance the clock
//     c.io.out.expect(0.U)  // Assert that the output correctly has 0
//     c.io.in.poke(1.U)     // Set our input to value 1
//     c.clock.step(1)    // advance the clock
//     c.io.out.expect(1.U)  // Assert that the output correctly has 1
//     c.io.in.poke(2.U)     // Set our input to value 2
//     c.clock.step(1)    // advance the clock
//     c.io.out.expect(2.U)  // Assert that the output correctly has 2
}

Elaborating design...
